In [76]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, classification_report, log_loss, make_scorer
import seaborn as sn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import sklearn
import warnings
warnings.filterwarnings("ignore")

### Part II

### Part III

Models.

Take this data. https://github.com/gSchool/dsi-logistic-regression/blob/g79/data/grad.csv

Predict whether someone will get into grad school. Use the following models.

- Logistic Regression
- Random Forest
- Naive Bayes (you will need to figure out what type works best for this data)

Which model performed the best?

In [77]:
df = pd.read_csv('grad.csv')
df.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [78]:
#df.info()

In [79]:
# class imbalance
#df.admit.sum() / 400, (df.admit.size - df.admit.sum()) / 400

In [80]:
# more admitting when rake 1 school vs rank 3 or 4
#pd.crosstab(df['admit'], df['rank'], rownames=['admit'])

In [81]:
y = df.pop('admit')
X = df

In [82]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=15)
scalar = StandardScaler().fit(X_train)
X_train = scalar.transform(X_train)
X_val = scalar.transform(X_val)

# Model 1:  Logistic Regression

In [83]:
params = {
          'penalty': ['l2'], 
          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
          'fit_intercept': [True, False],
          'C': np.logspace(-3,3)
         }
gs = GridSearchCV(LogisticRegression(), param_grid=params, scoring='neg_log_loss', cv=10, n_jobs=5).fit(X_train, y_train)
gs

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=5,
       param_grid={'penalty': ['l2'], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'fit_intercept': [True, False], 'C': array([1.00000e-03, 1.32571e-03, 1.75751e-03, 2.32995e-03, 3.08884e-03,
       4.09492e-03, 5.42868e-03, 7.19686e-03, 9.54095e-03, 1.26486e-02,
       1.67683e-02, 2.2230... 1.84207e+02, 2.44205e+02,
       3.23746e+02, 4.29193e+02, 5.68987e+02, 7.54312e+02, 1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [84]:
gs.best_params_

{'C': 0.15998587196060574,
 'fit_intercept': True,
 'penalty': 'l2',
 'solver': 'saga'}

In [85]:
model1 = LogisticRegression(**gs.best_params_).fit(X_train, y_train)

In [86]:
yhat = model1.predict(X_val)
phat = model1.predict_proba(X_val)[:,1]

In [87]:
print('Logistic Regression Score: ', accuracy_score(y_val, yhat))

Logistic Regression Score:  0.65


# Model 2:  Random Forest

In [88]:
params = {
            'n_estimators': [200],
            'criterion': ['gini', 'entropy'],
            'max_depth': range(1, 15),
            'max_features': ['sqrt', 'log2']
         }
gs = GridSearchCV(RandomForestClassifier(), param_grid=params, scoring='neg_log_loss', cv=5, n_jobs=-1).fit(X_train, y_train)
gs

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [200], 'criterion': ['gini', 'entropy'], 'max_depth': range(1, 15), 'max_features': ['sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [89]:
gs.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'log2',
 'n_estimators': 200}

In [90]:
medel2 = RandomForestClassifier(**gs.best_params_).fit(X_train, y_train)
yhat = medel2.predict(X_val)
phat = medel2.predict_proba(X_val)[:,1]

In [91]:
print('Random Forest Classifier Score:', accuracy_score(y_val, yhat))

Random Forest Classifier Score: 0.675


# Model 3:  Naive Bayes

In [92]:
model3 = GaussianNB().fit(X_train, y_train)
yhat = model3.predict(X_val)
phat = model3.predict_proba(X_val)[:,1]

In [93]:
print('Guassian NB Score: ', accuracy_score(y_val, yhat))

Guassian NB Score:  0.6833333333333333


## Part IV

Text Classification.

Remember this assignment.

https://github.com/data-science-ml/tweets-nlp-assignment/blob/master/nlp-assignment.md

Take the above tweets and turn them into a bag of words. Use a Naive Bayes classifier to figure out if a particular tweet is Neutral, Negative or Positive. Remember to split your data.

What is the accuracy of your model?

Compare this model to a KNN model (neighbors == 3) where each tweet is a 300 dimensional vector.

Which model performs better?

In [96]:
df1 = pd.read_csv('Tweets.csv')
df1.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)


In [97]:
text = df1.pop('text')
label = df1.pop('airline_sentiment')

In [99]:
len(text), len(label)

(14640, 14640)